In [1]:
import tensorflow as tf
import os
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from PIL import Image

In [2]:
print(tf.__version__)

2.10.0


In [3]:
batch_size=64
image_size=224
train_path='../data/original/archive/train'
test_path='../data/original/archive/test'

In [4]:
def image_resize(image):
    resized_image = tf.image.resize(image, size=(224, 224),method='lanczos5')
    return image

In [5]:

'''def image_resize_model(img):
    # Load the pre-trained ResNet50 model
    model = ResNet50(weights='imagenet')
    
    img_copy=img.copy()


    # Resize the image to a larger size
    img_resized = img_copy.resize((300, 300))

    # Preprocess the resized image
    img_preprocessed = preprocess_input(Image.img_to_array(img_resized))

    # Pass the preprocessed image to the ResNet50 model and get the resized image as output
    output = model.predict(img_preprocessed.reshape(1, 600, 600, 3))
    resized_image = Image.array_to_img(output[0])
    return resized_image'''

"def image_resize_model(img):\n    # Load the pre-trained ResNet50 model\n    model = ResNet50(weights='imagenet')\n    \n    img_copy=img.copy()\n\n\n    # Resize the image to a larger size\n    img_resized = img_copy.resize((300, 300))\n\n    # Preprocess the resized image\n    img_preprocessed = preprocess_input(Image.img_to_array(img_resized))\n\n    # Pass the preprocessed image to the ResNet50 model and get the resized image as output\n    output = model.predict(img_preprocessed.reshape(1, 600, 600, 3))\n    resized_image = Image.array_to_img(output[0])\n    return resized_image"

In [6]:
def image_processing(train_path,test_path,image_size,batch_size):
    train_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
                                    rescale=(1./255),
                                     shear_range=0.2,
                                     zoom_range=0.3, #try range
                                     width_shift_range=0.0,
                                     height_shift_range=0.0,
                                     brightness_range=[0.6,1.5],
                                     rotation_range=0.2,
                                     horizontal_flip=True) #try vertical
    
    train_gen=train_datagen.flow_from_directory(directory=train_path,
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 target_size=(image_size,image_size))
    
    test_datagen=ImageDataGenerator(rescale=(1./255))
     
    test_gen=test_datagen.flow_from_directory(directory=test_path,
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               target_size=(image_size,image_size))
    return train_gen,test_gen
                                    

In [7]:
train_data,test_data=image_processing(train_path,test_path,image_size,batch_size)

Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [8]:
'''for i in range(10):
    images,label=train_data.next()
    plt.imshow(images[0])
    print(label)
    plt.show()'''

'for i in range(10):\n    images,label=train_data.next()\n    plt.imshow(images[0])\n    print(label)\n    plt.show()'

In [9]:
class_names=train_data.class_indices
print(class_names)
class_num=len(class_names)
print(class_num)

{'Acne and Rosacea': 0, 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1, 'Atopic Dermatitis': 2, 'Bullous Disease': 3, 'Cellulitis Impetigo and other Bacterial Infections': 4, 'Eczema': 5, 'Exanthems and Drug Eruptions': 6, 'Hair Loss Alopecia and other Hair Diseases': 7, 'Herpes HPV and other STDs': 8, 'Light Diseases and Disorders of Pigmentation': 9, 'Lupus and other Connective Tissue diseases': 10, 'Melanoma Skin Cancer Nevi and Moles': 11, 'Nail Fungus and other Nail Disease': 12, 'Poison Ivy and other Contact Dermatitis': 13, 'Psoriasis,Lichen Planus and related diseases': 14, 'Scabies Lyme Disease and other Infestations and Bites': 15, 'Seborrheic Keratoses and other Benign Tumors': 16, 'Systemic Disease': 17, 'Tinea Ringworm Candidiasis and other Fungal Infections': 18, 'Urticaria Hives': 19, 'Vascular Tumors': 20, 'Vasculitis': 21, 'Warts Molluscum and other Viral Infections': 22}
23


In [10]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base_model=MobileNetV2(input_shape=(image_size,image_size,3),weights='imagenet')

In [11]:
for layer in base_model.layers[:-23]:
    layer.trainable = False

In [12]:
def output_of_last_layer(pre_trained_model, limit_layer):
    last_desired_layer = pre_trained_model.get_layer(limit_layer)
    print('last layer output shape: ', last_desired_layer.output_shape)
    last_output = last_desired_layer.output
    print('last layer output: ', last_output)

    return last_output

In [13]:
last_output = output_of_last_layer(base_model,'global_average_pooling2d')

last layer output shape:  (None, 1280)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='global_average_pooling2d/Mean:0', description="created by layer 'global_average_pooling2d'")


In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(class_num, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [15]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [ ]:
LR=0.01
EPOCHS=10

In [17]:
model.compile(Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    min_delta=0.001, 
    mode='min'
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
filepath = "best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.0001)
                        
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=EPOCHS, verbose=1,
                   callbacks=callbacks_list)



In [21]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
train_top2_acc = history.history['top_2_accuracy']
val_top2_acc = history.history['val_top_2_accuracy']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'r', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, train_top2_acc, 'r', label='Training top2 acc')
plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
plt.title('Training and validation top2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, train_top3_acc, 'r', label='Training top3 acc')
plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
plt.title('Training and validation top3 accuracy')
plt.legend()


plt.show()

NameError: name 'history' is not defined